# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    
    source_id_text = list()
    target_id_text = list()
    
    # convert the source text
    for sentence in source_text.split("\n"):
        source_sentence_numbers = [source_vocab_to_int[word] for word in sentence.split()]
        source_id_text.append(source_sentence_numbers)
    
    # convert the target text
    for sentence in target_text.split('\n'):
        target_sentence_numbers = [target_vocab_to_int[word] for word in sentence.split()]
        target_sentence_numbers.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(target_sentence_numbers)
    
    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0


/home/cbios/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    
    _input = tf.placeholder(shape=[None,None],name="input",dtype=tf.int32)
    _targets = tf.placeholder(shape=[None,None],name="targets",dtype=tf.int32)
    _learning_rate = tf.placeholder(shape=None,name="learning_rate",dtype=tf.float32)
    _keep_probability = tf.placeholder(shape=None,name="keep_prob",dtype=tf.float32)
    _target_sequence_length = tf.placeholder(shape=[None],name="target_sequence_length",dtype=tf.int32)
    _max_target_sequence_length = tf.reduce_max(_target_sequence_length,name="max_target_len")
    _source_sequence_length = tf.placeholder(shape=[None],name="source_sequence_length",dtype=tf.int32)
    
    return (_input, _targets, _learning_rate, _keep_probability, _target_sequence_length, _max_target_sequence_length, _source_sequence_length)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/home/cbios/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/home/cbios/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/home/cbios/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/home/cbios/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/home/cbios/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/home/cbios/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    
    # remove the last word id from each batch in target_data
    strided_slice = tf.strided_slice(target_data,
                                     begin=[0,0],
                                     end=[batch_size, -1],
                                     strides=[1,1])
    
    # concatenate the GO ID to the begining of each batch
    concat_values = tf.fill(dims=[batch_size, 1],
                    value=target_vocab_to_int['<GO>'])
    decoded_target_data = tf.concat(values=[concat_values,strided_slice],
                                   axis=1)
    
    
    return decoded_target_data
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # embed the encoder input using tf.contrib.layers.embed_sequence
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # construct a stacked tf.contrib.rnn.LSTMCell wrapped in a tf.contrib.rnn.DropoutWrapper using the num_layers
    lstm_layers = []
    for idx in range(num_layers):
        lstm_layers.append(tf.contrib.rnn.LSTMCell(rnn_size))
        
    cell = tf.contrib.rnn.DropoutWrapper( tf.contrib.rnn.MultiRNNCell(lstm_layers), keep_prob)
    
    # pass cell and embedded input to tf.nn.dynamic_rnn()
    rnn_output, rnn_state = tf.nn.dynamic_rnn(cell, embed, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return rnn_output, rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    
    # create a decoding layer using the training helper
    decoding_layer_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                       sequence_length=target_sequence_length)
    
    # create a basic decoder 
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,
                                                    helper=decoding_layer_helper, 
                                                    initial_state=encoder_state, 
                                                    output_layer=output_layer)
    
    # obtain the decoder outputs from tf.contrib.seq2seq.dynamic_decode
    decoder_outputs, decoder_state, decoder_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder=basic_decoder, 
                                                                                                 maximum_iterations=max_summary_length)
    
    return decoder_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    x:param encoder_state: Encoder state
    x:param dec_cell: Decoder RNN Cell
    x:param dec_embeddings: Decoder embeddings
    x:param start_of_sequence_id: GO ID
    x:param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    x:param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    
    # find the start_tokens and end tokens   
    # per the project slack channel/forums, tf.tile can help replicate the input multiples times.
    start_tokens = tf.tile(input=tf.constant([start_of_sequence_id],dtype=tf.int32),
                           multiples=[batch_size])
    
    
    
    # create a tf.contrib.seq2seq.GreedyEmbeddingHelper
    greedy_embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=dec_embeddings,
                                                                      start_tokens=start_tokens,
                                                                      end_token=end_of_sequence_id)
    
    # create a tf.contrib.seq2seq.BasicDecoder
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,
                                                    helper=greedy_embedding_helper,
                                                    initial_state=encoder_state,
                                                    output_layer=output_layer)
    
    # create the decoder outputs from tf.contrib.seq2seq.dynamic_decode
    basic_decoder_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder=basic_decoder,
                                                                                                  output_time_major=False,
                                                                                                  impute_finished=False,
                                                                                                  maximum_iterations=max_target_sequence_length,
                                                                                                  parallel_iterations=32,
                                                                                                  swap_memory=False,
                                                                                                  scope=None)
    
    return basic_decoder_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    x:param dec_input: Decoder input
    x:param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    x:param rnn_size: RNN Size
    x:param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    x:param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    x:param keep_prob: Dropout keep probability
    x:param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    
    start_of_sequence_id = source_vocab_to_int['<GO>']
    end_of_sequence_id = source_vocab_to_int['<EOS>']
    
    # embed the target sequences using tf.contrib.layers.embed_sequence
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    # looks up ids in a give tensor 
    dec_embed_input = tf.nn.embedding_lookup(params=dec_embeddings, 
                                             ids=dec_input)

    
    # construct a stacked tf.contrib.rnn.LSTMCell wrapped in a tf.contrib.rnn.DropoutWrapper using the num_layers
    lstm_layers = []
    for idx in range(num_layers):
        lstm_layers.append(tf.contrib.rnn.LSTMCell(rnn_size))    
    dec_cell = tf.contrib.rnn.DropoutWrapper( tf.contrib.rnn.MultiRNNCell(lstm_layers), keep_prob)
    
    # per the project 4 Slack channel, use dense with truncated normal init
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
     
    # use tf.variable_scope to share variables between training and inference.
    with tf.variable_scope('decoding', reuse=False) as decoding_scope:
   
        # get the training logits
        training_basic_decoder_output =  decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)
    
        # get the inference logits
        inference_basic_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, target_vocab_size, output_layer, batch_size, keep_prob)
    
    return training_basic_decoder_output, inference_basic_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # encode the input using your encoding_layer
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size) 
    rnn_output, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size, enc_embedding_size) 
    
    # process target data using process_decoder_input()
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)

    # decode in encoded input using decoding_layer()
    training_basic_decoder_output, inference_basic_decoder_output = decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
     
    return training_basic_decoder_output, inference_basic_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 300
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 25
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 13
decoding_embedding_size = 13
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = .5
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/269 - Train Accuracy: 0.2327, Validation Accuracy: 0.3096, Loss: 5.7543
Epoch   0 Batch   20/269 - Train Accuracy: 0.2386, Validation Accuracy: 0.3099, Loss: 5.5265
Epoch   0 Batch   30/269 - Train Accuracy: 0.2665, Validation Accuracy: 0.3096, Loss: 5.0690
Epoch   0 Batch   40/269 - Train Accuracy: 0.2437, Validation Accuracy: 0.3123, Loss: 4.6585
Epoch   0 Batch   50/269 - Train Accuracy: 0.2521, Validation Accuracy: 0.3123, Loss: 4.2811
Epoch   0 Batch   60/269 - Train Accuracy: 0.2920, Validation Accuracy: 0.3096, Loss: 3.8967
Epoch   0 Batch   70/269 - Train Accuracy: 0.2948, Validation Accuracy: 0.3220, Loss: 3.7458
Epoch   0 Batch   80/269 - Train Accuracy: 0.3157, Validation Accuracy: 0.3399, Loss: 3.6191
Epoch   0 Batch   90/269 - Train Accuracy: 0.2644, Validation Accuracy: 0.3410, Loss: 3.7033
Epoch   0 Batch  100/269 - Train Accuracy: 0.3199, Validation Accuracy: 0.3410, Loss: 3.4725
Epoch   0 Batch  110/269 - Train Accuracy: 0.3075, Validation Accuracy

Epoch   3 Batch  120/269 - Train Accuracy: 0.4509, Validation Accuracy: 0.4994, Loss: 2.1087
Epoch   3 Batch  130/269 - Train Accuracy: 0.4473, Validation Accuracy: 0.5049, Loss: 2.1278
Epoch   3 Batch  140/269 - Train Accuracy: 0.4866, Validation Accuracy: 0.5067, Loss: 1.9991
Epoch   3 Batch  150/269 - Train Accuracy: 0.4817, Validation Accuracy: 0.5065, Loss: 1.9845
Epoch   3 Batch  160/269 - Train Accuracy: 0.4883, Validation Accuracy: 0.5075, Loss: 1.9753
Epoch   3 Batch  170/269 - Train Accuracy: 0.4847, Validation Accuracy: 0.5060, Loss: 1.9547
Epoch   3 Batch  180/269 - Train Accuracy: 0.4844, Validation Accuracy: 0.5071, Loss: 1.9448
Epoch   3 Batch  190/269 - Train Accuracy: 0.4803, Validation Accuracy: 0.5044, Loss: 1.9242
Epoch   3 Batch  200/269 - Train Accuracy: 0.4645, Validation Accuracy: 0.5035, Loss: 1.9940
Epoch   3 Batch  210/269 - Train Accuracy: 0.4859, Validation Accuracy: 0.5062, Loss: 1.8970
Epoch   3 Batch  220/269 - Train Accuracy: 0.4982, Validation Accuracy

Epoch   6 Batch  230/269 - Train Accuracy: 0.3905, Validation Accuracy: 0.4256, Loss: 1.0697
Epoch   6 Batch  240/269 - Train Accuracy: 0.4658, Validation Accuracy: 0.4369, Loss: 0.9817
Epoch   6 Batch  250/269 - Train Accuracy: 0.4336, Validation Accuracy: 0.4776, Loss: 1.0821
Epoch   6 Batch  260/269 - Train Accuracy: 0.4196, Validation Accuracy: 0.4695, Loss: 1.0974
Epoch   7 Batch   10/269 - Train Accuracy: 0.4434, Validation Accuracy: 0.4721, Loss: 1.0677
Epoch   7 Batch   20/269 - Train Accuracy: 0.4154, Validation Accuracy: 0.4664, Loss: 1.0771
Epoch   7 Batch   30/269 - Train Accuracy: 0.4507, Validation Accuracy: 0.4743, Loss: 1.0199
Epoch   7 Batch   40/269 - Train Accuracy: 0.4618, Validation Accuracy: 0.4856, Loss: 1.0640
Epoch   7 Batch   50/269 - Train Accuracy: 0.4482, Validation Accuracy: 0.4885, Loss: 1.0643
Epoch   7 Batch   60/269 - Train Accuracy: 0.4893, Validation Accuracy: 0.4826, Loss: 0.9817
Epoch   7 Batch   70/269 - Train Accuracy: 0.4596, Validation Accuracy

Epoch  10 Batch   80/269 - Train Accuracy: 0.5608, Validation Accuracy: 0.5616, Loss: 0.7901
Epoch  10 Batch   90/269 - Train Accuracy: 0.5346, Validation Accuracy: 0.5722, Loss: 0.8459
Epoch  10 Batch  100/269 - Train Accuracy: 0.5759, Validation Accuracy: 0.5727, Loss: 0.7820
Epoch  10 Batch  110/269 - Train Accuracy: 0.5582, Validation Accuracy: 0.5720, Loss: 0.7898
Epoch  10 Batch  120/269 - Train Accuracy: 0.5288, Validation Accuracy: 0.5674, Loss: 0.8156
Epoch  10 Batch  130/269 - Train Accuracy: 0.5256, Validation Accuracy: 0.5668, Loss: 0.8365
Epoch  10 Batch  140/269 - Train Accuracy: 0.5635, Validation Accuracy: 0.5697, Loss: 0.8022
Epoch  10 Batch  150/269 - Train Accuracy: 0.5479, Validation Accuracy: 0.5724, Loss: 0.8026
Epoch  10 Batch  160/269 - Train Accuracy: 0.5685, Validation Accuracy: 0.5726, Loss: 0.7896
Epoch  10 Batch  170/269 - Train Accuracy: 0.5764, Validation Accuracy: 0.5760, Loss: 0.7865
Epoch  10 Batch  180/269 - Train Accuracy: 0.5710, Validation Accuracy

Epoch  13 Batch  190/269 - Train Accuracy: 0.6057, Validation Accuracy: 0.6029, Loss: 0.6751
Epoch  13 Batch  200/269 - Train Accuracy: 0.5938, Validation Accuracy: 0.6022, Loss: 0.7099
Epoch  13 Batch  210/269 - Train Accuracy: 0.6059, Validation Accuracy: 0.6066, Loss: 0.6741
Epoch  13 Batch  220/269 - Train Accuracy: 0.6063, Validation Accuracy: 0.6024, Loss: 0.6499
Epoch  13 Batch  230/269 - Train Accuracy: 0.5995, Validation Accuracy: 0.6040, Loss: 0.6843
Epoch  13 Batch  240/269 - Train Accuracy: 0.6361, Validation Accuracy: 0.6126, Loss: 0.6302
Epoch  13 Batch  250/269 - Train Accuracy: 0.5752, Validation Accuracy: 0.6050, Loss: 0.7035
Epoch  13 Batch  260/269 - Train Accuracy: 0.5736, Validation Accuracy: 0.6143, Loss: 0.7217
Epoch  14 Batch   10/269 - Train Accuracy: 0.5924, Validation Accuracy: 0.6155, Loss: 0.7021
Epoch  14 Batch   20/269 - Train Accuracy: 0.5945, Validation Accuracy: 0.6015, Loss: 0.7023
Epoch  14 Batch   30/269 - Train Accuracy: 0.6118, Validation Accuracy

...

Epoch 284 Batch   40/269 - Train Accuracy: 0.8523, Validation Accuracy: 0.8721, Loss: 0.1191
Epoch 284 Batch   50/269 - Train Accuracy: 0.8558, Validation Accuracy: 0.8735, Loss: 0.1123
Epoch 284 Batch   60/269 - Train Accuracy: 0.8829, Validation Accuracy: 0.8752, Loss: 0.0911
Epoch 284 Batch   70/269 - Train Accuracy: 0.8844, Validation Accuracy: 0.8693, Loss: 0.1043
Epoch 284 Batch   80/269 - Train Accuracy: 0.8747, Validation Accuracy: 0.8754, Loss: 0.1050
Epoch 284 Batch   90/269 - Train Accuracy: 0.8783, Validation Accuracy: 0.8731, Loss: 0.1073
Epoch 284 Batch  100/269 - Train Accuracy: 0.8829, Validation Accuracy: 0.8751, Loss: 0.0988
Epoch 284 Batch  110/269 - Train Accuracy: 0.8616, Validation Accuracy: 0.8709, Loss: 0.1011
Epoch 284 Batch  120/269 - Train Accuracy: 0.8772, Validation Accuracy: 0.8705, Loss: 0.1001
Epoch 284 Batch  130/269 - Train Accuracy: 0.8635, Validation Accuracy: 0.8727, Loss: 0.1010
Epoch 284 Batch  140/269 - Train Accuracy: 0.8639, Validation Accuracy

Epoch 287 Batch  150/269 - Train Accuracy: 0.8712, Validation Accuracy: 0.8710, Loss: 0.1078
Epoch 287 Batch  160/269 - Train Accuracy: 0.8820, Validation Accuracy: 0.8699, Loss: 0.1012
Epoch 287 Batch  170/269 - Train Accuracy: 0.8686, Validation Accuracy: 0.8763, Loss: 0.0990
Epoch 287 Batch  180/269 - Train Accuracy: 0.8930, Validation Accuracy: 0.8747, Loss: 0.0941
Epoch 287 Batch  190/269 - Train Accuracy: 0.8783, Validation Accuracy: 0.8860, Loss: 0.0997
Epoch 287 Batch  200/269 - Train Accuracy: 0.8610, Validation Accuracy: 0.8776, Loss: 0.0933
Epoch 287 Batch  210/269 - Train Accuracy: 0.8660, Validation Accuracy: 0.8784, Loss: 0.1069
Epoch 287 Batch  220/269 - Train Accuracy: 0.8749, Validation Accuracy: 0.8711, Loss: 0.0917
Epoch 287 Batch  230/269 - Train Accuracy: 0.8687, Validation Accuracy: 0.8794, Loss: 0.0943
Epoch 287 Batch  240/269 - Train Accuracy: 0.8837, Validation Accuracy: 0.8736, Loss: 0.0876
Epoch 287 Batch  250/269 - Train Accuracy: 0.8806, Validation Accuracy

Epoch 290 Batch  260/269 - Train Accuracy: 0.8552, Validation Accuracy: 0.8675, Loss: 0.1142
Epoch 291 Batch   10/269 - Train Accuracy: 0.8639, Validation Accuracy: 0.8612, Loss: 0.1060
Epoch 291 Batch   20/269 - Train Accuracy: 0.8624, Validation Accuracy: 0.8755, Loss: 0.1027
Epoch 291 Batch   30/269 - Train Accuracy: 0.8676, Validation Accuracy: 0.8701, Loss: 0.0996
Epoch 291 Batch   40/269 - Train Accuracy: 0.8543, Validation Accuracy: 0.8725, Loss: 0.1184
Epoch 291 Batch   50/269 - Train Accuracy: 0.8585, Validation Accuracy: 0.8720, Loss: 0.1031
Epoch 291 Batch   60/269 - Train Accuracy: 0.8817, Validation Accuracy: 0.8776, Loss: 0.0953
Epoch 291 Batch   70/269 - Train Accuracy: 0.8784, Validation Accuracy: 0.8653, Loss: 0.0942
Epoch 291 Batch   80/269 - Train Accuracy: 0.8730, Validation Accuracy: 0.8714, Loss: 0.0978
Epoch 291 Batch   90/269 - Train Accuracy: 0.8768, Validation Accuracy: 0.8678, Loss: 0.1024
Epoch 291 Batch  100/269 - Train Accuracy: 0.8876, Validation Accuracy

Epoch 294 Batch  110/269 - Train Accuracy: 0.8539, Validation Accuracy: 0.8670, Loss: 0.1060
Epoch 294 Batch  120/269 - Train Accuracy: 0.8729, Validation Accuracy: 0.8717, Loss: 0.1056
Epoch 294 Batch  130/269 - Train Accuracy: 0.8587, Validation Accuracy: 0.8771, Loss: 0.0990
Epoch 294 Batch  140/269 - Train Accuracy: 0.8601, Validation Accuracy: 0.8648, Loss: 0.1051
Epoch 294 Batch  150/269 - Train Accuracy: 0.8705, Validation Accuracy: 0.8733, Loss: 0.0974
Epoch 294 Batch  160/269 - Train Accuracy: 0.8781, Validation Accuracy: 0.8675, Loss: 0.1068
Epoch 294 Batch  170/269 - Train Accuracy: 0.8583, Validation Accuracy: 0.8673, Loss: 0.0946
Epoch 294 Batch  180/269 - Train Accuracy: 0.8874, Validation Accuracy: 0.8683, Loss: 0.0930
Epoch 294 Batch  190/269 - Train Accuracy: 0.8822, Validation Accuracy: 0.8723, Loss: 0.0961
Epoch 294 Batch  200/269 - Train Accuracy: 0.8634, Validation Accuracy: 0.8738, Loss: 0.0939
Epoch 294 Batch  210/269 - Train Accuracy: 0.8704, Validation Accuracy

Epoch 297 Batch  220/269 - Train Accuracy: 0.8703, Validation Accuracy: 0.8692, Loss: 0.0976
Epoch 297 Batch  230/269 - Train Accuracy: 0.8642, Validation Accuracy: 0.8757, Loss: 0.1030
Epoch 297 Batch  240/269 - Train Accuracy: 0.8854, Validation Accuracy: 0.8743, Loss: 0.0835
Epoch 297 Batch  250/269 - Train Accuracy: 0.8782, Validation Accuracy: 0.8773, Loss: 0.1019
Epoch 297 Batch  260/269 - Train Accuracy: 0.8687, Validation Accuracy: 0.8768, Loss: 0.1088
Epoch 298 Batch   10/269 - Train Accuracy: 0.8715, Validation Accuracy: 0.8656, Loss: 0.0897
Epoch 298 Batch   20/269 - Train Accuracy: 0.8630, Validation Accuracy: 0.8767, Loss: 0.0974
Epoch 298 Batch   30/269 - Train Accuracy: 0.8683, Validation Accuracy: 0.8683, Loss: 0.0950
Epoch 298 Batch   40/269 - Train Accuracy: 0.8581, Validation Accuracy: 0.8780, Loss: 0.1178
Epoch 298 Batch   50/269 - Train Accuracy: 0.8490, Validation Accuracy: 0.8750, Loss: 0.1116
Epoch 298 Batch   60/269 - Train Accuracy: 0.8825, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [24]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    list_of_word_ids = list()
    
    # convert sentence to lowercase
    lower_case_sentence = sentence.lower()
    print(lower_case_sentence)
    
    # convert words into ids using vocab_to_int
    for word in lower_case_sentence.split():
        if word in vocab_to_int.keys():
            list_of_word_ids.append(vocab_to_int[word])
        else:
            list_of_word_ids.append(vocab_to_int['<UNK>'])
    return (list_of_word_ids)
                                
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

this is a test sentence
Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [25]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


he saw a old yellow truck .
INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [190, 25, 202, 93, 197, 56, 215]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [211, 183, 262, 303, 53, 356, 197, 1]
  French Words: poire à chaude est brillante leur ? <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.